In [0]:
%sql
USE workspace.default;
DROP TABLE IF EXISTS churn_limpo;
DROP TABLE IF EXISTS churn_silver

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.churn_zero.churn_silver AS
SELECT 
    *,
    
    -- 1. Sentimento
    ai_analyze_sentiment(CustomerFeedback) as sentiment_label,
    
    -- 2. Tópico (Labels ajustados para precisão)
    ai_classify(
        CustomerFeedback, 
        ARRAY(
            'high_price',              -- Custos/Preço alto
            'service_quality',         -- Qualidade técnica/sinal
            'internet_connection',     -- Sem internet/Lentidão
            'billing_issues',          -- Cobrança errada
            'competitor_offer',        -- Competição
            'contract_terms',          -- Problemas com contrato/Fidelidade
            'personal_reasons',        -- Mudança de casa/Circunstâncias pessoais
            'customer_service',        -- Atendimento ruim
            'general_satisfaction',    -- Elogios/Neutro (IMPORTANTE para falsos positivos)
            'other'
        )
    ) as feedback_topic,

    -- 3. Score Numérico
    CASE 
        WHEN ai_analyze_sentiment(CustomerFeedback) = 'positive' THEN 1 
        ELSE 0 
    END as sentiment_score

FROM workspace.churn_zero.churn_limpo;

In [0]:
%sql
-- Validação da Distribuição dos Tópicos
SELECT 
    feedback_topic,
    count(*) as total
FROM workspace.churn_zero.churn_silver
WHERE churn = 1 AND sentiment_score = 1
GROUP BY feedback_topic
ORDER BY total DESC;

In [0]:
%sql
SELECT
 *
FROM workspace.churn_zero.churn_silver

In [0]:
df = spark.sql("SELECT * FROM workspace.churn_zero.churn_silver")
df.toPandas().to_csv("/Workspace/Users/joaovab@al.insper.edu.br/Hackathon_Terra_Signal_grupo_1/churn_silver.csv")